# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [2]:
import os
import random
import time
import sys
from msvcrt import kbhit, getch

# Game constants
WIDTH = 40
HEIGHT = 20
GAME_DURATION = 120
SNAKE_SPEED = 0.1  # Seconds between moves

# ANSI escape codes
COLOR_BLUE = '\033[94m'
COLOR_GREEN = '\033[92m'
COLOR_RED = '\033[91m'
COLOR_YELLOW = '\033[93m'
COLOR_RESET = '\033[0m'
CLEAR_SCREEN = '\033[2J'
HIDE_CURSOR = '\033[?25l'
SHOW_CURSOR = '\033[?25h'

def move_cursor(x, y):
    print(f'\033[{y+1};{x+1}H', end='')

class SnakeGame:
    def __init__(self):
        self.game_over = False
        self.score = 0
        self.x = WIDTH // 2
        self.y = HEIGHT // 2
        self.fruit_x = random.randint(0, WIDTH-1)
        self.fruit_y = random.randint(0, HEIGHT-1)
        self.tail_x = []
        self.tail_y = []
        self.direction = 'STOP'
        self.start_time = time.time()
        self.last_move_time = time.time()

    def clear_screen(self):
        print(CLEAR_SCREEN, end='')

    def draw_border(self):
        print(COLOR_BLUE + '#' * (WIDTH + 2))
        for _ in range(HEIGHT):
            print('#' + ' ' * WIDTH + '#')
        print('#' * (WIDTH + 2) + COLOR_RESET)

    def draw(self):
        move_cursor(0, 0)

        # Draw border
        print(COLOR_BLUE + '#' * (WIDTH + 2))
        for i in range(HEIGHT):
            print('#', end='')
            for j in range(WIDTH):
                if i == self.y and j == self.x:
                    print(COLOR_GREEN + 'O' + COLOR_RESET, end='')
                elif i == self.fruit_y and j == self.fruit_x:
                    print(COLOR_RED + 'F' + COLOR_RESET, end='')
                else:
                    tail_printed = False
                    for k in range(len(self.tail_x)):
                        if self.tail_x[k] == j and self.tail_y[k] == i:
                            print(COLOR_GREEN + 'o' + COLOR_RESET, end='')
                            tail_printed = True
                            break
                    if not tail_printed:
                        print(' ', end='')
            print(COLOR_BLUE + '#')
        print('#' * (WIDTH + 2) + COLOR_RESET)

        # Draw UI
        elapsed = time.time() - self.start_time
        remaining = max(GAME_DURATION - int(elapsed), 0)
        print(f"{COLOR_YELLOW}Time left: {remaining} seconds{COLOR_RESET}")
        print(f"Score: {self.score}")
        print("Controls: W/A/S/D to move, ESC to quit")

    def handle_input(self):
        if not kbhit():
            return

        key = getch().decode()
        new_dir = self.direction

        if key == 'a' or key == 'A':
            new_dir = 'LEFT'
        elif key == 'd' or key == 'D':
            new_dir = 'RIGHT'
        elif key == 'w' or key == 'W':
            new_dir = 'UP'
        elif key == 's' or key == 'S':
            new_dir = 'DOWN'
        elif key == '\x1b':  # ESC
            self.game_over = True

        # Prevent 180-degree turns
        if (new_dir == 'LEFT' and self.direction != 'RIGHT' or
            new_dir == 'RIGHT' and self.direction != 'LEFT' or
            new_dir == 'UP' and self.direction != 'DOWN' or
            new_dir == 'DOWN' and self.direction != 'UP'):
            self.direction = new_dir

    def update(self):
        # Move tail
        if len(self.tail_x) > 0:
            self.tail_x.insert(0, self.x)
            self.tail_y.insert(0, self.y)
            self.tail_x.pop()
            self.tail_y.pop()

        # Move head
        if self.direction == 'LEFT':
            self.x -= 1
        elif self.direction == 'RIGHT':
            self.x += 1
        elif self.direction == 'UP':
            self.y -= 1
        elif self.direction == 'DOWN':
            self.y += 1

        # Wrap around
        self.x %= WIDTH
        self.y %= HEIGHT

        # Check collisions
        for i in range(len(self.tail_x)):
            if self.tail_x[i] == self.x and self.tail_y[i] == self.y:
                self.game_over = True

        # Check fruit collision
        if self.x == self.fruit_x and self.y == self.fruit_y:
            self.score += 10
            valid = False
            while not valid:
                self.fruit_x = random.randint(0, WIDTH-1)
                self.fruit_y = random.randint(0, HEIGHT-1)
                valid = True
                if self.fruit_x == self.x and self.fruit_y == self.y:
                    valid = False
                for i in range(len(self.tail_x)):
                    if self.fruit_x == self.tail_x[i] and self.fruit_y == self.tail_y[i]:
                        valid = False
                        break
            self.tail_x.append(0)
            self.tail_y.append(0)

    def game_over_screen(self):
        self.clear_screen()
        move_cursor(WIDTH//2 - 5, HEIGHT//2 - 2)
        print(COLOR_RED + "GAME OVER!" + COLOR_RESET)
        move_cursor(WIDTH//2 - 8, HEIGHT//2)
        print(f"Final Score: {self.score}")
        move_cursor(WIDTH//2 - 12, HEIGHT//2 + 2)
        print("Press any key to exit...")
        getch()

    def run(self):
        print(HIDE_CURSOR)
        try:
            while not self.game_over:
                self.clear_screen()

                if time.time() - self.start_time > GAME_DURATION:
                    self.game_over = True
                    break

                self.handle_input()

                if time.time() - self.last_move_time >= SNAKE_SPEED and self.direction != 'STOP':
                    self.update()
                    self.last_move_time = time.time()

                self.draw()
                time.sleep(0.05)

            self.game_over_screen()
        finally:
            print(SHOW_CURSOR)

if __name__ == "__main__":
    game = SnakeGame()
    game.run()


##########################################
#                                        #
#                                        #
#                                        #
#                                        #
#                                        #
#                                        #
#                                        #
#                                        #
#                                        #
#                                        #
#                    O                   #
#                                        #
#                                        #
#                                        #
#                                        #
#                       F                #
#                                        #
#                                        #
#                                        #
#                                        #
##########################################
Time left: 120 seconds
Score: 0
Controls: W/A/S/D to 

KeyboardInterrupt: 